In [1]:
# !pip install spacy

from dotenv import load_dotenv
import ast

load_dotenv()

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andriimyronenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

test_tech_df = pd.read_csv('./data/test_df_tech_only_combined.csv')

test_tech_df['knowledge'] = test_tech_df['knowledge'].apply(ast.literal_eval)
test_tech_df['skills'] = test_tech_df['skills'].apply(ast.literal_eval)

In [3]:
print(test_tech_df.at[0, 'text'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'skills'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'knowledge'])

Full Stack Software Engineer - Java / JavaScript <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> . <ADDRESS> <ADDRESS> <LOCATION> - <LOCATION> Date posted: 2021-03-04 Likes: 0 Dislikes: 0 Love: 0 Salary: <SALARY> Job type: FULL_TIME Experience level: <EXPERIENCE> Industry: <INDUSTRY> Company size: <SIZE> Company type: <COMPANY_TYPE> Technologies: javascript reactjs java Job description: Job type: Full-time Role: Full Stack Developer Technologies javascript reactjs java Job description As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices . You'll be required to apply your depth of knowledge and expertise to all aspects of the software development lifecycle as well as partner continuously with your many stakeholders on a daily basis to stay focused on common goals . We embrace a culture of experimentation and constantly strive for i

In [4]:
test_tech_df.shape

(31, 5)

In [5]:
verbose_template = """
You are a top-notch recruiter and data labeler.
Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills or knowledge.

You should use the following guideline for annotation:

Instruction for Annotating Text for Skills and Knowledge Components
Objective: Identify and annotate spans of text in job postings (JPs) that represent specific skills or knowledge required for the position.

1. Identifying Skills:

General Rule: A skill is usually indicated by a VERB or an (ADJECTIVE) + NOUN combination.
Exclusion of Modal Verbs: Do not tag modal verbs (e.g., can, will) as part of the skill.
Phrase Separation: Split phrases containing prepositions and/or conjunctions, except when conjunctions coordinate two nouns as a single argument.
Handling Anaphoric Pronouns: Avoid tagging skills with anaphoric pronouns. Only tag the preceding skill.
Splitting Coordinated Components: Split nouns and adjectives in coordination if they lack a verb.
Listing of Skills: If skills are listed leading to different subtasks, annotate each separately.
Brevity in Contextual Information: Keep skill annotations concise, especially when followed by company-specific info.
Inclusion of 'skills' or 'knowledge' in Tags: Include these words in the annotation if their omission alters the meaning.
Parenthetical Information: Include if it elaborates on the skill or is an abbreviation.
Adverbial Inclusion: Include adverbials that describe the manner of doing something.
Attitudes as Skills: Annotate attitudes as skills, omitting articles.
Exclusions:
Avoid tagging ironic skills.
Avoid nested annotations; use one span for overlapping skills.
Do not tag skills in top headlines, but do tag in sub-headlines and body text.
Exclude fluff and triggers surrounding the skill component.
Exclude participation, contributing, and transfer expressions from skill annotations.
Do not annotate occupations or positions.
Focus on skills related and specific to the position.

2. Identifying Knowledge:
General Rule: Knowledge is non-executable and possessed by an individual.
Inclusion of Parenthetical Information: Include if it is related to the knowledge component.
Licenses and Certifications: Include additional words like "certificate," "license," etc.
Vague Preceding Verbs: Only tag the knowledge component if preceded by a vague verb.
Specificity: Annotate only specified knowledge components.
Nested Knowledge in Skills: Knowledge components can be nested within skill annotations.
Coordination of Knowledge Components: Annotate as one if all components share one knowledge tag.
Listing Knowledge Tags: Annotate all knowledge tags separately.
Annotation of Industries and Fields: Tag these as knowledge components.

3. Prioritization and Other Considerations:
Prefer skills over knowledge in uncertain cases.
Prioritize skills over attitudes; only tag the skill within an attitude.
Keep annotations concise and relevant to the job.
Annotate skills and knowledge in unconventional places if related to the position.
Consider annotating a combination of skill and knowledge when applicable.
Exclude knowledge/skill components in positions.
Focus on annotations relevant to the specific position and its future expectations.

Output just the JSON array, wihtout any other text. 
Start your response with "[" symbol, and finish with "]".
Make sure your response doesn't include anything else.

Job posting:
{job_posting}

Response:"""

short_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills and knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.

    Output the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include anything else.
        
    Job posting:
    {job_posting}
"""

few_shot_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including job-related skills or knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.
    
    Below you can see some examples. Pay attention to what is included in detected spans. We want not to include false positive skills and knowledge.

    Job posting:
    Ability to work in large collaborative teams to achieve organizational goals
    Detected spans:
    ["work in large collaborative teams"]

    Job posting:
    Work hands-on together with the other engineers within the Agile team 
    Detected spans:
    ["Work hands-on"]
    
    Job posting:
    Requirements At least 5 years of combined experience in Java or Kotlin and JavaScript or TypeScript programming and related test frameworks ( Selenium TestCafe etc.) .
    Detected spans:
    ["Java", "Kotlin", "JavaScript", "Typescript"]
    
    Job posting:
    A degree in Computer Science or related fields or equivalent practical experience . 
    Detected spans:
    ["degree in Computer Science"]
    
    Job posting:
    Experience in working on a cloud-based application running on Docker .
    Detected spans:
    ["Docker", "working on a cloud-based application"]
    Make sure to never include protected attributes like gender identity or veteran status in the returned list:
    
    Job posting:
    We do not discriminate on the basis of any protected attribute including race religion color national origin gender sexual orientation .
    Detected spans:
    []

    Output just the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include data from examples and anything else except from JSON array.
    Return empty array If there are no obvious skill and knowledge spans in the given posting.
    
    Job posting:
    {job_posting}
    Detected spans:
"""

annotation_guidelines_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including job-related skills and knowledge components.
    The definition of skill is the following:
    "The ability to apply knowledge and use know-how to complete tasks and solve problems. Skills are described as cognitive (involving the use of logical, intuitive and creative thinking) or practical (involving manual dexterity and the use of methods, materials, tools and instruments)."
    The definition of knowledge is the following:
    "The body of facts, principles, theories and practices that is related to a field of work or study. Knowledge is described as theoretical and/or factual, and is the outcome of the assimilation of information through learning."
    Make sure to detect the shortest possible text span excluding irrelevant, company-specific information. Avoid including general descriptions or knowledge areas, and concentrate on specific abilities or competencies required to perform job tasks effectively.
    
    Output just the JSON array, with skill or knowledge strings surrounded by quotation marks. 
    Start your response with "[" symbol, and finish with "]".
    
    Example job posting including skill components:
    We are looking for people who want to work in cross-functional teams and want to help others learn how to build them    
    Detected spans:
    ["work in cross-functional teams", "help others learn"]
    
    Example job posting including knowledge components:
    Experience with latest versions of  and     
    Detected spans:
    ["Java", "Spring"]
    
    Make sure your response doesn't include anything else except from JSON array.
    Return empty array If there are no obvious skill or knowledge components in the given posting.
    
    Job posting:
    {job_posting}
    Detected spans:
"""


template_collection = {
    'verbose': verbose_template,
    'short': short_template,
    'few_shot': few_shot_template,
    'annotation_guidelines_template': annotation_guidelines_template
}

In [6]:
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(input_variables=["job_posting"], template=template_collection['annotation_guidelines_template'])

In [7]:
from langchain.llms.fireworks import Fireworks 

llm = Fireworks(model="accounts/fireworks/models/llama-v2-70b-chat", max_tokens=-1, top_p=0.6)

In [8]:
import ast

test_posting = test_tech_df.at[0, 'text']

def get_skills_for_posting(job_posting):
    prompt_val = prompt_template.invoke({"job_posting": job_posting})
    prompt = prompt_val.to_string()
    output = llm(prompt)
    array_start = output.find('[')
    array_end = output.find(']', array_start)
    array_string = output[array_start:array_end+1].strip()
    return ast.literal_eval(array_string)
     
llama_results = get_skills_for_posting(test_posting)
llama_results

['work in cross-functional teams',
 'partner continuously with stakeholders',
 'apply depth of knowledge and expertise',
 'build technology that clients will pay for',
 'create commercial opportunities',
 'BS/BA degree or equivalent experience',
 'advanced knowledge of application data and infrastructure architecture disciplines',
 'understanding of architecture and design across all systems',
 'working proficiency in developmental toolsets',
 'knowledge of industry-wide technology strategies and best practices',
 'ability to work in large collaborative teams',
 'proficiency in one or more modern programming languages',
 'understanding of software skills such as business analysis development maintenance and software improvement',
 'in-depth HTML5 / JavaScript',
 'competent with developing multithreaded scalable concurrent applications',
 'Node JS (server side and tooling)',
 'React or similar view framework']

In [9]:
def calculate_f1_score(real_values, predicted_values):
    real_values = [val.lower() for val in real_values]
    predicted_values = [val.lower() for val in predicted_values]
 
    tp = sum([1 for val in predicted_values if val in real_values])
    fp = sum([1 for val in predicted_values if val not in real_values])
    fn = sum([1 for val in real_values if val not in predicted_values])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))

    return f1_score

In [10]:
merged_skills_knowledge = test_tech_df.at[0, 'skills'] + test_tech_df.at[0, 'knowledge']

merged_skills_knowledge

['solving business problems',
 'apply your depth of knowledge and expertise',
 'partner continuously with your many stakeholders',
 'stay focused on common goals',
 'motivated',
 'deliver of a next-generation analytics products',
 'developmental toolsets',
 'work in large collaborative teams',
 'achieve organizational goals',
 'building an innovative culture',
 'developing multithreaded scalable concurrent applications',
 'javascript',
 'reactjs',
 'java',
 'javascript',
 'reactjs',
 'java',
 'innovation and engineering practices',
 'software development lifecycle',
 'BS/BA degree',
 'application data and infrastructure architecture disciplines',
 'architecture',
 'design',
 'industry-wide technology strategies and best practices',
 'Java',
 'Javascript',
 'business analysis',
 'development',
 'maintenance',
 'software improvement',
 'HTML5',
 'JavaScript',
 'Node JS ( server side and tooling ) *',
 'React']

In [11]:
calculate_f1_score(merged_skills_knowledge, llama_results)

Precision: 0.0
Recall: 0.0


0

In [12]:
from nltk.tokenize import sent_tokenize

def divide_into_chunks(text, min_sentences=3):
    sentences = sent_tokenize(text)
    
    chunks = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(current_chunk) == min_sentences:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

chunks = divide_into_chunks(test_posting)

In [13]:
def get_skills_for_posting_chunked(job_posting):
    chunks = divide_into_chunks(job_posting)
    responses = []
    
    for chunk in chunks:
        try:
            prompt_val = prompt_template.invoke({"job_posting": chunk})
            prompt = prompt_val.to_string()
            output = llm(prompt)
            array_start = output.find('[')
            array_end = output.find(']', array_start)

            array_string = output[array_start:array_end+1].strip()
            responses = responses + ast.literal_eval(array_string)
        except:
            print("ERROR processing chunk")
    
    return responses
    
responses = get_skills_for_posting_chunked(test_posting)

responses

['Full Stack Developer',
 'Full Stack Software Engineer',
 'javascript',
 'reactjs',
 'java',
 'software development lifecycle',
 'innovation',
 'engineering practices',
 'solving business problems',
 'partner',
 'stakeholders',
 'common goals',
 'work in a collaborative',
 'trusting',
 'thought-provoking',
 'diversity of thought',
 'creative solutions',
 'next-generation analytics',
 'institutional client base',
 'build technology',
 'create commercial opportunities',
 'shape the key components of the product',
 'fellow technologists',
 'build a modern and comprehensive API',
 'allow clients to easily define and compute analytics',
 'BS/BA degree',
 'application data and infrastructure architecture disciplines',
 'architecture and design',
 'developmental toolsets',
 'industry-wide technology strategies',
 'modern programming languages',
 'business analysis development maintenance',
 'software improvement',
 'HTML5',
 'JavaScript',
 'multithreaded scalable concurrent applications',
 '

In [14]:
calculate_f1_score(merged_skills_knowledge, responses)

Precision: 0.24
Recall: 0.41379310344827586


0.3037974683544304

In [15]:
f1_scores = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        res = get_skills_for_posting(row['text'])
        f1_score = calculate_f1_score(row_merged_skills_knowledge, res)
        print(f1_score)
        f1_scores.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")
        
f1_scores

Precision: 0.0
Recall: 0.0
0
Precision: 0.625
Recall: 0.35714285714285715
0.45454545454545453
Precision: 0.2727272727272727
Recall: 0.04477611940298507
0.07692307692307691
Precision: 0.07142857142857142
Recall: 0.025
0.03703703703703704
Precision: 0.3333333333333333
Recall: 0.08
0.12903225806451613
Precision: 0.8461538461538461
Recall: 0.2972972972972973
0.44000000000000006
Precision: 0.45454545454545453
Recall: 0.2777777777777778
0.3448275862068966
Precision: 0.375
Recall: 0.33962264150943394
0.3564356435643565
Precision: 0.3333333333333333
Recall: 0.47368421052631576
0.391304347826087
Precision: 0.2
Recall: 0.04878048780487805
0.07843137254901962
Failed to parse skills for row 10: invalid syntax (<unknown>, line 1)
Precision: 0.42857142857142855
Recall: 0.07894736842105263
0.13333333333333333
Precision: 0.5
Recall: 0.15789473684210525
0.23999999999999996
Precision: 0.3333333333333333
Recall: 0.15789473684210525
0.21428571428571427
Precision: 0.5555555555555556
Recall: 0.2272727272727

[0,
 0.45454545454545453,
 0.07692307692307691,
 0.03703703703703704,
 0.12903225806451613,
 0.44000000000000006,
 0.3448275862068966,
 0.3564356435643565,
 0.391304347826087,
 0.07843137254901962,
 0.13333333333333333,
 0.23999999999999996,
 0.21428571428571427,
 0.3225806451612903,
 0.3793103448275862,
 0.08888888888888889,
 0.40740740740740744,
 0.04166666666666667,
 0.21917808219178084,
 0,
 0.18867924528301885,
 0.18518518518518517,
 0.07692307692307691,
 0.14492753623188406,
 0.45833333333333326,
 0.2456140350877193,
 0.12698412698412698,
 0.03389830508474576]

In [16]:
sum(f1_scores) / len(f1_scores)

0.20770473941400722